In [1]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB URI if different
db = client["TransitGuide_TrainDatabase"]  # Database name
collection = db["Train_Database"]  # Collection name

# Query the data
for document in collection.find():
    print(document)


In [15]:
import requests
import json
import psycopg2
from psycopg2 import sql
from datetime import datetime
import random

# PostgreSQL Configuration
POSTGRES_HOST = "localhost"
POSTGRES_DB = "Train_Database"
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "root"
POSTGRES_TABLE = "train_data"  # Table to insert data into

# API Configuration
API_URL = "https://railways.makemytrip.com/api/tbsWithAvailabilityAndRecommendation/NDLS/BCT/20241214"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Price mapping for classes with randomization
def get_class_price(class_name):
    """Return a random price for a given class."""
    price_mapping = {
        "2S": random.randint(325, 435),  # Fixed price
        "SL": random.randint(1000, 1200),  # Slight variation around the base price
        "CC": random.randint(1700, 1900),
        "3A": random.randint(2400, 2600),
        "3E": random.randint(2400, 2600),
        "2A": random.randint(3100, 3300),
        "1A": random.randint(3800, 4000)
    }
    return price_mapping.get(class_name, 0)  # Default to 0 if class not found

def fetch_api_data():
    """Fetch train details from API and assign prices based on class."""
    try:
        response = requests.get(API_URL, headers=HEADERS)
        response.raise_for_status()
        data = response.json()
        trains = data.get("trainBtwnStnsList", [])
        extracted_trains = []
        for train in trains:
            # Extract available classes
            classes = []
            if "tbsAvailability" in train:
                classes = [availability.get("className", "N/A") for availability in train.get("tbsAvailability", [])]

            # Create a row for each class
            for cls in classes:
                extracted_trains.append({
                    "train_number": train.get("trainNumber"),
                    "train_name": train.get("trainName"),
                    "from_station_code": train.get("frmStnCode"),
                    "from_station_city": train.get("frmStnCity"),
                    "to_station_code": train.get("toStnCode"),
                    "to_station_city": train.get("toStnCity"),
                    "price": get_class_price(cls),  # Assign price based on class
                    "class": cls,  # Class column
                    "date": "2024-12-14"  # Fixed date
                })
        return extracted_trains
    except requests.exceptions.RequestException as e:
        print(f"Error fetching API data: {e}")
        return []

def create_postgres_table():
    """Create a table in PostgreSQL if it doesn't exist."""
    conn = psycopg2.connect(
        host=POSTGRES_HOST,
        database=POSTGRES_DB,
        user=POSTGRES_USER,
        password=POSTGRES_PASSWORD,
    )
    cur = conn.cursor()
    create_table_query = sql.SQL("""
        CREATE TABLE IF NOT EXISTS {table} (
            id SERIAL PRIMARY KEY,
            train_number TEXT,
            train_name TEXT,
            from_station_code TEXT,
            from_station_city TEXT,
            to_station_code TEXT,
            to_station_city TEXT,
            price INTEGER,
            class TEXT,
            date DATE
        )
    """).format(table=sql.Identifier(POSTGRES_TABLE))
    cur.execute(create_table_query)
    conn.commit()
    cur.close()
    conn.close()

def insert_data_to_postgres(data):
    """Insert data into PostgreSQL."""
    conn = psycopg2.connect(
        host=POSTGRES_HOST,
        database=POSTGRES_DB,
        user=POSTGRES_USER,
        password=POSTGRES_PASSWORD,
    )
    cur = conn.cursor()
    insert_query = sql.SQL("""
        INSERT INTO {table} (
            train_number, train_name, from_station_code, 
            from_station_city, to_station_code, to_station_city, price, class, date
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """).format(table=sql.Identifier(POSTGRES_TABLE))

    for record in data:
        try:
            cur.execute(insert_query, (
                record["train_number"],
                record["train_name"],
                record["from_station_code"],
                record["from_station_city"],
                record["to_station_code"],
                record["to_station_city"],
                record["price"],  # Price based on randomization
                record["class"],
                record["date"]
            ))
        except Exception as e:
            print(f"Error inserting record: {record} -> {e}")

    conn.commit()
    cur.close()
    conn.close()


if __name__ == "__main__":
    # Step 1: Fetch data from API
    api_data = fetch_api_data()

    if api_data:
        # Step 2: Ensure the PostgreSQL table exists
        create_postgres_table()

        # Step 3: Insert data into PostgreSQL
        insert_data_to_postgres(api_data)

        print("Data transfer from API to PostgreSQL is complete!")
    else:
        print("No data fetched from API.")


Data transfer from API to PostgreSQL is complete!


In [16]:
import requests
import json

# API Endpoint
url = "https://railways.makemytrip.com/api/tbsWithAvailabilityAndRecommendation/NDLS/BCT/20241214"

# Headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

try:
    # Make a GET request
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

    # Parse JSON response
    data = response.json()

    # Extract required train details
    trains = data.get("trainBtwnStnsList", [])  # Adjust the key based on the actual API response

    extracted_trains = []
    for train in trains:
        # Extract available classes
        classes = []
        if "tbsAvailability" in train:
            classes = [availability.get("className", "N/A") for availability in train.get("tbsAvailability", [])]

        # Extract train details
        train_info = {
            "trainNumber": train.get("trainNumber"),
            "trainName": train.get("trainName"),
            "fromStationCode": train.get("frmStnCode"),
            "fromStationCity": train.get("frmStnCity"),
            "toStationCode": train.get("toStnCode"),
            "toStationCity": train.get("toStnCity"),
            "price": train.get("minPriceDetails", {}).get("amount", "N/A"),
            "classes": classes
        }
        extracted_trains.append(train_info)

    # Print extracted details
    print(json.dumps(extracted_trains, indent=4))

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


[
    {
        "trainNumber": "12952",
        "trainName": "Mmct Tejas Raj",
        "fromStationCode": "NDLS",
        "fromStationCity": "Delhi",
        "toStationCode": "MMCT",
        "toStationCity": "Mumbai",
        "price": "N/A",
        "classes": [
            "3A",
            "2A",
            "1A"
        ]
    },
    {
        "trainNumber": "12248",
        "trainName": "Bandra Yuva Exp",
        "fromStationCode": "NZM",
        "fromStationCity": "Delhi",
        "toStationCode": "BDTS",
        "toStationCity": "Mumbai",
        "price": "N/A",
        "classes": [
            "3A",
            "CC"
        ]
    },
    {
        "trainNumber": "22414",
        "trainName": "Nzm Mao Rajdani",
        "fromStationCode": "NZM",
        "fromStationCity": "Delhi",
        "toStationCode": "PNVL",
        "toStationCity": "Mumbai",
        "price": "N/A",
        "classes": [
            "3A",
            "2A",
            "1A"
        ]
    },
    {
        "trainNum